# CURSO DE FUNDAMENTOS DE PYTHON  
## Base de Datos con **SQLite** desde Python

**Objetivos**
- Conectar a SQLite con `sqlite3` (sin instalar nada extra).
- Crear esquema y realizar operaciones **CRUD** con consultas **parametrizadas**.
- Usar **transacciones**, manejo de **excepciones** y **pandas** para análisis.
- Validar con `assert` que las operaciones se ejecutaron correctamente.

### 0) Preparación del entorno

In [1]:
# No requiere instalación: 'sqlite3' viene con la librería estándar de Python.
import sqlite3, os, time
from pathlib import Path

DB_PATH = Path.cwd() / "sqlite_demo.db"
if DB_PATH.exists():
    DB_PATH.unlink()  # Reiniciar base para prácticas reproducibles

conn = sqlite3.connect(DB_PATH)  # crea el archivo si no existe
conn.execute("PRAGMA foreign_keys = ON;")
conn.row_factory = sqlite3.Row  # para acceder por nombre de columna
print("Conectado a:", DB_PATH)

Conectado a: e:\Archivos (1)\sqlite_demo.db


### 1) Crear esquema

In [2]:
with conn:
    conn.execute("""
    CREATE TABLE alumnos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        carrera TEXT NOT NULL,
        nota REAL CHECK(nota BETWEEN 0 AND 10),
        creado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
print("Tabla 'alumnos' creada.")

Tabla 'alumnos' creada.


### 2) INSERT (parametrizado)

In [3]:
alumnos = [
    ("Ana", "Big Data", 9.2),
    ("Luis", "Desarrollo", 7.8),
    ("María", "Big Data", 8.9),
    ("Pedro", "Ciberseguridad", 6.5),
    ("Julia", "Big Data", 9.8),
]
with conn:
    conn.executemany(
        "INSERT INTO alumnos(nombre, carrera, nota) VALUES (?, ?, ?)",
        alumnos
    )
total = conn.execute("SELECT COUNT(*) AS c FROM alumnos").fetchone()["c"]
print("Insertados:", total)

Insertados: 5


### 3) SELECT básicos

In [4]:
# Todos
for row in conn.execute("SELECT id, nombre, carrera, nota FROM alumnos"):
    print(dict(row))

# Filtrados
print("\nSolo Big Data con nota >= 9:")
sql = "SELECT nombre, nota FROM alumnos WHERE carrera=? AND nota>=? ORDER BY nota DESC"
for row in conn.execute(sql, ("Big Data", 9.0)):
    print(dict(row))

{'id': 1, 'nombre': 'Ana', 'carrera': 'Big Data', 'nota': 9.2}
{'id': 2, 'nombre': 'Luis', 'carrera': 'Desarrollo', 'nota': 7.8}
{'id': 3, 'nombre': 'María', 'carrera': 'Big Data', 'nota': 8.9}
{'id': 4, 'nombre': 'Pedro', 'carrera': 'Ciberseguridad', 'nota': 6.5}
{'id': 5, 'nombre': 'Julia', 'carrera': 'Big Data', 'nota': 9.8}

Solo Big Data con nota >= 9:
{'nombre': 'Julia', 'nota': 9.8}
{'nombre': 'Ana', 'nota': 9.2}


### 4) UPDATE y DELETE (parametrizados)

In [5]:
with conn:
    conn.execute("UPDATE alumnos SET nota = ? WHERE nombre = ?", (8.0, "Pedro"))
    conn.execute("DELETE FROM alumnos WHERE nombre = ?", ("Luis",))
r = conn.execute("SELECT COUNT(*) AS c FROM alumnos WHERE nombre='Luis'").fetchone()["c"]
print("¿Existe Luis?", r == 1)  # False esperado

¿Existe Luis? False


### 5) Transacciones (COMMIT / ROLLBACK)

In [6]:
try:
    conn.isolation_level = None  # modo manual
    conn.execute("BEGIN;")
    conn.execute("INSERT INTO alumnos(nombre, carrera, nota) VALUES (?, ?, ?)", ("Temporal", "Big Data", 5.0))
    # Simular error para forzar ROLLBACK
    raise RuntimeError("Fallo simulado")
    conn.execute("COMMIT;")
except Exception as e:
    print("Error:", e, "-> ROLLBACK")
    conn.execute("ROLLBACK;")
finally:
    conn.isolation_level = ""  # vuelve al modo autocommit por defecto

existe = conn.execute("SELECT COUNT(*) AS c FROM alumnos WHERE nombre='Temporal'").fetchone()["c"]
print("Insert temporal persistió?", existe > 0)  # False esperado

Error: Fallo simulado -> ROLLBACK
Insert temporal persistió? False


### 6) Manejo de excepciones

In [7]:
try:
    with conn:
        # Violará el CHECK de nota
        conn.execute("INSERT INTO alumnos(nombre, carrera, nota) VALUES (?, ?, ?)", ("Error", "Big Data", 11.5))
except sqlite3.IntegrityError as e:
    print("Excepción capturada:", e.__class__.__name__)

Excepción capturada: IntegrityError


### 7) Consultas con agregación y GROUP BY

In [8]:
sql = "SELECT carrera, COUNT(*) AS n, ROUND(AVG(nota),2) AS prom FROM alumnos GROUP BY carrera ORDER BY prom DESC"
for row in conn.execute(sql):
    print(dict(row))

{'carrera': 'Big Data', 'n': 3, 'prom': 9.3}
{'carrera': 'Ciberseguridad', 'n': 1, 'prom': 8.0}


### 8) Índices y EXPLAIN QUERY PLAN

In [9]:
with conn:
    conn.execute("CREATE INDEX IF NOT EXISTS idx_alumnos_carrera ON alumnos(carrera);")
plan = conn.execute("EXPLAIN QUERY PLAN " + "SELECT * FROM alumnos WHERE carrera='Big Data'").fetchall()
print("PLAN:", plan[:3], "...")

PLAN: [<sqlite3.Row object at 0x000001654858EDA0>] ...


### 9) Integración con `pandas` (opcional)

In [10]:
# Si no tienes pandas, instala con: !pip install pandas
try:
    import pandas as pd
    df = pd.read_sql_query("SELECT id, nombre, carrera, nota FROM alumnos", conn)
    display(df.head())
except Exception as e:
    print("Pandas no disponible:", e)

C:\Users\59399\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,id,nombre,carrera,nota
0,1,Ana,Big Data,9.2
1,3,María,Big Data,8.9
2,4,Pedro,Ciberseguridad,8.0
3,5,Julia,Big Data,9.8


### 10) Cierre de conexión

In [11]:
conn.close()
print("Conexión cerrada.")

Conexión cerrada.


### ✅ Prácticas con validación (`assert`)

In [12]:
# Re-abrimos conexión solo para validaciones
conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row

# 1) Implementa una función que devuelva el promedio por carrera
def promedio_por_carrera(carrera):
    row = conn.execute(
        "SELECT AVG(nota) AS prom FROM alumnos WHERE carrera=?",
        (carrera,)
    ).fetchone()
    return round(row["prom"], 2) if row["prom"] is not None else None

# Tests
p_bigdata = promedio_por_carrera("Big Data")
assert 8.5 <= p_bigdata <= 10.0
assert promedio_por_carrera("NoExiste") is None

# 2) Verifica que no exista 'Luis' después del DELETE
c_luis = conn.execute("SELECT COUNT(*) AS c FROM alumnos WHERE nombre='Luis'").fetchone()["c"]
assert c_luis == 0

conn.close()
print("Validaciones OK ✅")

Validaciones OK ✅
